In [1]:
from officelib.xllib import *

In [2]:
xl = Excel()
wb = xl.Workbooks.Open("C:\\Users\\nstar\\Documents\\Dropbox\\LogMeIn Info\\Product Configuration unlocked.xlsx")

In [51]:
class Sheet():
    def __init__(self, wb, Name):
        self.wb = wb
        self.xl = wb.Application
        self.ws = wb.Worksheets(Name)
        self.cells = self.ws.Cells
        self.cr = self.cells.Range

class Data():
    def __init__(self, sheet, tbl_name):
        self.sheet = sheet
        self.tbl_name = tbl_name
        self.tbl = self.sheet.ws.ListObjects(self.tbl_name)
        self.data = self.tbl.DataBodyRange.Value
        
        col = self.tbl.HeaderRowRange.Value[0]
        self.columns = {name: i for i, name in enumerate(col)}
        
    def rowq(self, row, column):
        return row[self.columns[column]]
    

In [52]:
ws_br = Sheet(wb, "Bioreactors")
ws_cust = Sheet(wb, "Customer Info")
ws_cont = Sheet(wb, "Contacts")

In [53]:
data_br = Data(ws_br, "BioreactorDatabase")
data_cust = Data(ws_cust, "CustomerInfo")
data_cont = Data(ws_cont, "Contacts")

In [54]:
def rowq(row, column):
    return row[columns[column]].Value

def br_needs_notification(row):
    version = data_br.rowq(row, "Software Version")
    return version is not None and version.startswith("3.0")

def company_from_row(row):
    return data_br.rowq(row, "Division/Group")

def contact(row):
    company = data_cont.rowq(row, "Division/Group")
    first = data_cont.rowq(row, "First Name")
    last = data_cont.rowq(row, "Last Name")
    email = data_cont.rowq(row, "Email address")
    return (company, first, last, email)

def should_notify(row):
    return data_cont.rowq(row, "Division/Group") in companies_to_notify

brs_to_update = filter(br_needs_notification, data_br.data)
companies_to_notify = set(map(company_from_row, brs_to_update))

contacts = sorted(set(map(contact, filter(should_notify, data_cont.data))))

In [72]:
out_wb = xl.Workbooks.Add()
out_ws = Sheet(out_wb, "Sheet1")

headers = [("Company", "Contact Name", "Email Address", "Completed By", "Date", "Comments")]
h1 = out_ws.cr("A1")
h2 = h1.GetOffset(0, len(headers[0])-1)
r1 = out_ws.cr(h1, h2)
r1.Value2 = headers

# contacts_data_out = [(company, f"{fn} {ln}", email) for company, fn, ln, email in contacts]

def make_contacts_data(contacts):
    lastc = ""
    lastr = []
    namel = []
    emaill = []
    out = []
    for c, fn, ln, em in contacts:
        if c != lastc:
            namel = []
            emaill = []
            lastr = [c, namel, emaill]
            out.append(lastr)
            lastc = c
        namel.append(f"{fn} {ln}")
        emaill.append(em if em is not None else "<none>")
    
    # stringify
    out2 =[]
    for c, l1, l2 in out:
        out2.append((c, "\n".join(l1), "\n".join(l2)))
    return out2

contacts_data_out = make_contacts_data(contacts)

def off(cr, c1, c2, n):
    c3 = c1.GetOffset(0, n)
    c4 = c2.GetOffset(0, n)
    return cr(c3, c4)

with screen_lock(xl):
    c1 = out_ws.cr("A2")
    c2 = c1.GetOffset(len(contacts_data_out) - 1, len(contacts_data_out[0]) - 1)
    out_ws.cr(c1, c2).Value = contacts_data_out

#     fc = c = c1
#     company = c.Value2
#     while True:
#         nc = c.GetOffset(1,0)
#         cm = nc.Value2
#         if cm != company:
#             if c != fc:
#                 out_ws.cr(fc,c).Merge()
#                 for i in range(1, 6):  # for each column other than the first
#                     off(out_ws.cr, fc, c, i).Merge()
                
#             fc = nc
#             company = cm
#             if cm is None:
#                 break
#         c = nc
    
    ur = out_ws.ws.UsedRange
    for col in ur.Columns:
        col.EntireColumn.ColumnWidth = 255 # widen first to prevent wrap on autofit
        col.EntireColumn.AutoFit()
    for row in ur.Rows:
        row.EntireRow.AutoFit()

In [68]:
r1.Address

'$A$1:$A$6'

In [71]:
r1.Value2 = headers[0]